# Florence Nightingale's Dataset
Modern hospital care and nursing can be said to have begun with the English statistician and nurse [Florence Nightingale](https://en.wikipedia.org/wiki/Florence_Nightingale).  During the [Crimean War](https://en.wikipedia.org/wiki/Crimean_War), Ms. Nightingale tended to wounded troops, and invented modern hospital practices: primarily, sanitation and disinfection.  To show the efficacy of her techniques, she counted deaths due to disease, wounds, and undetermined causes, showing that disease was the primary killer in war.  She showed that after her reforms, deaths dropped dramatically, and deaths due to disease more so.

The original dataset is taken from [Nightingale's Rose](https://github.com/datasets-io/nightingales-rose/), and is copyright the [Compute.io](https://github.com/compute-io) authors.

In [ ]:
import pandas as pd
from galyleo.galyleo_table import GalyleoTable
from galyleo.galyleo_jupyterlab_client import GalyleoClient

Read the original data 

In [ ]:
nightingale = pd.read_json('https://raw.githubusercontent.com/datasets-io/nightingales-rose/master/lib/dataset.json')

Extract the month from the date, and set so month 1 is April 1854, Month 24 is march 1856, then get rid of the spurious date column

In [ ]:
nightingale['month'] = nightingale['date'].apply(lambda date: (date.month - 3) + 12 * (date.year - 1854))
nightingale = nightingale.iloc[:, 1:]

Compute the total deaths each month, and then use these to compute the percentage of deaths due to each cause.

In [ ]:
nightingale['total'] = nightingale['disease'] + nightingale['wounds'] + nightingale['other']
def compute_pct(column_name):
    nightingale[f'{column_name}_pct'] = nightingale[column_name]/nightingale['total'] * 100
for column in columns: compute_pct(column)

Make and send the nightingale table (as summary) to the dashboard

In [ ]:
table = GalyleoTable('summary')
table.load_from_dataframe(nightingale)
client = GalyleoClient()
client.send_data_to_dashboard(table)

Compute the detail table.  This has columns (month, deaths, cause).  This is used for the pie charts above the main line charts.

In [ ]:
from functools import reduce
# Compute the table (month, death, cause) for each cause
def extract_cause_table(cause):
    slice = nightingale[['month', cause]].rename(columns = {cause: 'deaths'})
    slice['cause'] = cause
    return slice
# form the cause tables
cause_tables = [extract_cause_table(column) for column in columns]
# use reduce to merge them into a single table
merged = reduce(lambda acc, cur: acc.merge(cur, how = "outer"), cause_tables[1:], cause_tables[0])
# Use this to create the detail table and send to the dashboard
detail = GalyleoTable('detail')
detail.load_from_dataframe(merged)
client.send_data_to_dashboard(detail)